# Testing the OLLAMA API

In [2]:
%pip install requests

]11;?\Requirement already satisfied: requests in ./.venv/lib64/python3.13/site-packages (2.32.3)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import json

In [4]:
url = "http://localhost:11434/api/generate"

headers = {
    "Content-Type": "application/json"
}

data = {
    "model" : "llama3.2",
    "prompt" : "Hello, My name is Daniel Adnan",
    "stream" : False,
}

response = requests.post(url, headers=headers, data=json.dumps(data))

if response.status_code == 200:
    response_text = response.text
    data = json.loads(response_text)
    actual_response = data["response"]
    print(actual_response)
else: 
    print("Error: ", response.status_code, response.text)

Hello Daniel! It's nice to meet you. Is there something I can help you with or would you like to chat?


## Adding memory to the model

By default OLLAMA does not preserve memory

In [5]:
data = {
    "model" : "llama3.2",
    "prompt" : "What is my name?",
    "stream" : False,
}

response = requests.post(url, headers=headers, data=json.dumps(data))

if response.status_code == 200:
    response_text = response.text
    data = json.loads(response_text)
    actual_response = data["response"]
    print(actual_response)
else: 
    print("Error: ", response.status_code, response.text)

I don't have that information. I'm a large language model, I don't have the ability to know your personal details or keep track of individual users. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to share your name with me, I can certainly address you by that name if you'd like!


Adding Memory

In [6]:
%pip install ollama

]11;?\Requirement already satisfied: ollama in ./.venv/lib64/python3.13/site-packages (0.4.4)
Note: you may need to restart the kernel to use updated packages.


In [7]:

from ollama import chat as ollama_chat

model = 'llama3.2'
messages = []
# Roles
USER = 'user'
ASSISTANT = 'assistant'

def add_history(content, role):
    messages.append({'role': role, 'content': content})

In [8]:
def chat(message):
    add_history(message, USER)
    response = ollama_chat(model=model, messages=messages, stream=False)
    complete_message = ''
    for line in response:
        # Check if the line is a tuple and contains the 'message' key
        if isinstance(line, tuple) and line[0] == 'message':
            message_content = line[1].content
            complete_message += message_content
            # print(message_content, end='', flush=True)
        # else:
        #     print("Unexpected line format:", line)
    add_history(complete_message, ASSISTANT)
    return complete_message

In [9]:
chat_response = chat("Hello, my name is Shadab")
print(chat_response)

Hi Shadab! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [10]:
chat_response = chat("What is my name?")
print(chat_response)

You told me your name earlier - it's Shadab! How can I assist you today, Shadab?


In [11]:
messages = []
chat_response = chat("What is my name?")
print(chat_response)
print(messages)
messages = []

I don't have any information about you, including your name. This conversation just started, and I'm a large language model, I don't retain any personal data or information about individual users. Each time you interact with me, it's a new conversation, and I don't have any prior knowledge about you. Would you like to introduce yourself?
[{'role': 'user', 'content': 'What is my name?'}, {'role': 'assistant', 'content': "I don't have any information about you, including your name. This conversation just started, and I'm a large language model, I don't retain any personal data or information about individual users. Each time you interact with me, it's a new conversation, and I don't have any prior knowledge about you. Would you like to introduce yourself?"}]


# Working on RAG

Getting the necessary libraries

In [12]:
%pip install transformers datasets torch faiss-cpu matplotlib scikit-learn

]11;?\Requirement already satisfied: transformers in ./.venv/lib64/python3.13/site-packages (4.47.0)
Note: you may need to restart the kernel to use updated packages.


Add imports section

In [13]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import torch
import numpy as np
import random
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np

/home/shadab/Desktop/Github Repos/RAG with OLLAMA/.venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# suppress warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

## Loading and preprocessing data

Import the pdf file


In [15]:

%pip install PyMuPDF

]11;?\Requirement already satisfied: PyMuPDF in ./.venv/lib64/python3.13/site-packages (1.25.1)
Note: you may need to restart the kernel to use updated packages.


In [16]:
import fitz  

# Open the PDF file
pdf_document = "random_story.pdf"
document = fitz.open(pdf_document)

all_text = ""

for page_num in range(len(document)):
    page = document.load_page(page_num) 
    text = page.get_text()  
    all_text += text 

print(all_text)

Once upon a time, in a small village nestled on the banks of the mighty Padma River in Bangladesh, lived a boy named Arif. The village, called 
Balukandi, was a picturesque place where lush green rice paddies stretched endlessly, and the gentle hum of nature was a constant 
companion. Arif, a spirited twelve-year-old, was known for his curious mind and boundless energy.
 
Arif’s family was not wealthy, but they were rich in love and traditions. His father, Rahim Mia, was a fisherman who spent long hours on the river, 
casting his net in hopes of a bountiful catch. His mother, Amina Begum, managed their small household and worked tirelessly in their vegetable 
garden. Despite their modest means, they ensured that Arif attended the local school, which was a short walk from their home.
 
Every morning, after saying his prayers and helping his mother fetch water from the village well, Arif would grab his worn-out satchel and head 
to school. The path to school was one of his favorite parts

Process the text (splitting by paragraph)

In [17]:
# Split the text into paragraphs (simple split by newline characters)
def read_and_split_text(all_text):
    
    paragraphs = all_text.split('\n')
    paragraphs = [para.strip() for para in paragraphs if len(para.strip()) > 0]
    return paragraphs


# Split the text into paragraphs
paragraphs = read_and_split_text(all_text)

for i in range(4):
    print(f"sample: {i} paragraph: {paragraphs[i]} \n" )


sample: 0 paragraph: Once upon a time, in a small village nestled on the banks of the mighty Padma River in Bangladesh, lived a boy named Arif. The village, called 

sample: 1 paragraph: Balukandi, was a picturesque place where lush green rice paddies stretched endlessly, and the gentle hum of nature was a constant 

sample: 2 paragraph: companion. Arif, a spirited twelve-year-old, was known for his curious mind and boundless energy. 

sample: 3 paragraph: Arif’s family was not wealthy, but they were rich in love and traditions. His father, Rahim Mia, was a fisherman who spent long hours on the river, 



## Embedding

Tokenize the text

In [18]:
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
context_tokenizer

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


DPRContextEncoderTokenizer(name_or_path='facebook/dpr-ctx_encoder-single-nq-base', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [19]:
text = paragraphs[0]
print (text)

tokens_result=context_tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=256)
tokens_result

Once upon a time, in a small village nestled on the banks of the mighty Padma River in Bangladesh, lived a boy named Arif. The village, called


{'input_ids': tensor([[  101,  2320,  2588,  1037,  2051,  1010,  1999,  1037,  2235,  2352,
         22704,  2006,  1996,  5085,  1997,  1996, 10478, 23731,  2314,  1999,
          7269,  1010,  2973,  1037,  2879,  2315, 10488,  2546,  1012,  1996,
          2352,  1010,  2170,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Encoding into vector embeddings

In [20]:
context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
outputs=context_encoder(**tokens_result)
outputs

DPRContextEncoderOutput(pooler_output=tensor([[ 4.0093e-01,  4.2511e-02, -2.9443e-01,  9.9187e-02,  3.7222e-01,
          5.6648e-01,  2.2446e-01, -1.0303e-01,  8.5561e-02, -7.3476e-01,
         -1.7684e-01, -3.4557e-01, -3.9370e-01,  6.4019e-01,  2.8712e-01,
          1.1167e-01,  3.9522e-01,  2.3242e-01, -3.3190e-01, -2.9161e-01,
         -7.0793e-01,  5.1561e-02,  3.4618e-01,  3.1245e-01,  7.7873e-01,
         -6.1256e-02,  2.3302e-01, -6.3971e-02, -1.4170e-02,  1.0972e-01,
          9.8490e-02,  3.3719e-01,  2.3089e-01, -5.8903e-01, -7.6483e-01,
         -2.7139e-01, -1.3550e-01,  1.7743e-01,  4.9149e-02, -7.8392e-01,
         -2.0239e-01, -4.1871e-01,  4.0637e-01,  6.4429e-02, -3.0704e-01,
         -7.6216e-01, -9.4178e-01,  6.6036e-01, -3.7666e-01, -1.3964e-01,
          3.0464e-01,  5.4273e-01,  1.9491e-01, -5.3098e-01,  2.5661e-01,
          4.5500e-01, -4.4324e-01,  5.0109e-02,  1.2105e-01, -5.4375e-01,
          1.3316e+00,  8.7864e-01,  1.0550e+00,  4.1776e-01, -8.7826e-02,


Function to tokenize and embed the input text from PDF

In [22]:
def encode_contexts(text_list):
    embeddings = []
    for text in text_list:
        inputs = context_tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=256)
        outputs = context_encoder(**inputs)
        embeddings.append(outputs.pooler_output)
    return torch.cat(embeddings).detach().numpy()

random.shuffle(paragraphs)

context_embeddings = encode_contexts(paragraphs)

# store the dimenstion of the vector embeddings
paragraphs_column = context_embeddings.shape[1]
print(paragraphs_column)

768


## Indexing (with FAISS)

In [23]:
import faiss

# Convert list of numpy arrays into a single numpy array
embedding_dim = paragraphs_column 
context_embeddings_np = np.array(context_embeddings).astype('float32')

# Create a FAISS index for the embeddings
index = faiss.IndexFlatL2(embedding_dim)
index.add(context_embeddings_np)  # Add the context embeddings to the index

## Question Encoder & Tokenizer

Load DPR question encoder and tokenizer

In [24]:
question_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Encoding and tokenizing sample question

In [25]:
question = 'Who is Arif?'
question_inputs = question_tokenizer(question, return_tensors='pt')
question_embedding = question_encoder(**question_inputs).pooler_output.detach().numpy()

Search context from input PDF file

In [26]:
# Search the index
D, I = index.search(question_embedding, k=5)  # Retrieve top 5 relevant contexts
print("D:",D)
print("I:",I)

print("Top 5 relevant contexts:")
for i, idx in enumerate(I[0]):
    print(f"{i+1}: {paragraphs[idx]}")
    print(f"distance {D[0][i]}\n")

D: [[ 80.44499  86.9055   89.54919  93.60104 101.62082]]
I: [[18  5 21  4 24]]
Top 5 relevant contexts:
1: Arif’s family was not wealthy, but they were rich in love and traditions. His father, Rahim Mia, was a fisherman who spent long hours on the river,
distance 80.44499206542969

2: Arif’s story spread beyond Balukandi. Journalists from the town came to interview him, and he even received an invitation to a science
distance 86.90550231933594

3: Balukandi became a symbol of hope, and Arif’s journey inspired countless others to believe in the power of dreams and determination. And
distance 89.54918670654297

4: Rahim Mia chuckled and said, “It’s expensive, my boy. Besides, we’ve always lived with lanterns. Why change now?”
distance 93.60104370117188

5: But Arif wasn’t convinced. That night, he lay on his straw mattress, staring at the dim flicker of the oil lamp, and made up his mind. He would
distance 101.62081909179688



Function to search context from question

In [27]:
def search_relevant_contexts(question, question_tokenizer, question_encoder, index, k=20): # return top 5 relevant contexts
    # Tokenize the question
    question_inputs = question_tokenizer(question, return_tensors='pt')

    # Encode the question to get the embedding
    question_embedding = question_encoder(**question_inputs).pooler_output.detach().numpy()

    # Search the index to retrieve top k relevant contexts
    D, I = index.search(question_embedding, k)

    return D, I


# Test the function
question = "What is the name of father of Arif?"
D, I = search_relevant_contexts(question, question_tokenizer, question_encoder, index, k=5)

print("Distances:", D)
print("Indices:", I)

Distances: [[ 73.19446   83.350555  95.97923   96.402756 101.032196]]
Indices: [[18 21 24  5  4]]


## Integrating OLLAMA

Function to generate an answer using OLLAMA

In [28]:
def chat(message):
    add_history(message, USER)
    response = ollama_chat(model=model, messages=messages, stream=False)
    complete_message = ''
    for line in response:
        # Check if the line is a tuple and contains the 'message' key
        if isinstance(line, tuple) and line[0] == 'message':
            message_content = line[1].content
            complete_message += message_content
            # print(message_content, end='', flush=True)
        # else:
        #     print("Unexpected line format:", line)
    add_history(complete_message, ASSISTANT)
    return complete_message

def generate_answer_with_ollama(question, relevant_contexts):
    context_text = " ".join(relevant_contexts)
    prompt = f"Context: {context_text}\n\nQuestion: {question}\nAnswer:"
    response = chat(prompt)
    return response

Test the function

In [34]:
question = "Can you summarize the story?"
D, I = search_relevant_contexts(question, question_tokenizer, question_encoder, index, k=20)

relevant_contexts = [paragraphs[i] for i in I[0]]

# print the relevant contexts
for i, context in enumerate(relevant_contexts):
    print(f"{i+1}: {context}\n")

answer = generate_answer_with_ollama(question, relevant_contexts)

if answer:
    print(answer)

else:
    print("No answer found")

1: Rahim Mia chuckled and said, “It’s expensive, my boy. Besides, we’ve always lived with lanterns. Why change now?”

2: project, involving other village children in the effort. They scavenged materials, built turbines, and even learned basic wiring.

3: The day he tested his creation, half the village gathered to watch. With his friends spinning the turbine blades, the dynamo began to hum, and a

4: couldn’t hide their tears of pride.

5: Months later, on a cool winter evening, the village of Balukandi experienced a moment of magic. The community center lit up with the glow of

6: Balukandi became a symbol of hope, and Arif’s journey inspired countless others to believe in the power of dreams and determination. And

7: tiny bulb flickered to life. The crowd erupted in cheers. Although it was a small step, it was proof that even a village boy could dream big.

8: community center and the mosque. The elders, initially skeptical, eventually agreed. With their blessing and some donations,